In [ ]:
import csv
import re
from pathlib import Path

import pandas
import matplotlib.pyplot as plt

In [ ]:
base_path = Path("/data/incoming/")

Note on columns:
- `v` is value in thousand USD.
- `q` is quantity in metric tons.
- `hs` codes are strings (can start with zero, if they get mangled into integers they can be left-zero-padded to recover the n-digit string, and can extract shorter codes from the first n digits of the longer code). There are different editions every few years. Assuming these are HS2017.

In [ ]:
df = pandas.read_csv(
    base_path
    / "verschuur-2024-baci-mode-prediction"
    / "baci_mode_prediction_2020_HS6.csv",
    usecols=[
        "iso3_O",
        "iso3_D",
        "v",
        "q",
        "hs6",
        "hs4",
        "hs2",
        "v_air_predict",
        "v_sea_predict",
        "v_land_predict",
        "q_air_predict",
        "q_sea_predict",
        "q_land_predict",
    ],
    dtype={
        "hs6": "str",
        "hs4": "str",
        "hs2": "str",
    },
)
df

In [ ]:
df.hs6 = df.hs6.str.zfill(6)
df.hs4 = df.hs4.str.zfill(4)
df.hs2 = df.hs2.str.zfill(2)

In [ ]:
# df = df.query("iso3_O == 'THA' | iso3_D == 'THA'")

In [ ]:
! cd {base_path / "BACI"} && wget -q -nc https://unstats.un.org/unsd/classifications/Econ/tables/CPC/CPCv21_HS2017/CPC21-HS2017.csv
! cd {base_path / "BACI"} && wget -q -nc https://unstats.un.org/unsd/classifications/Econ/tables/ISIC/ISIC4_CPCv21/isic4-cpc21.txt
! cd {base_path / "BACI"} && wget -q -nc https://unstats.un.org/unsd/classifications/Econ/Download/In%20Text/ISIC_Rev_4_english_structure.Txt

In [ ]:
codes_cpc21_hs17 = pandas.read_csv(
    base_path / "BACI" / "CPC21-HS2017.csv", dtype="str"
).rename(columns={"HS 2017": "hs17_6", "CPC Ver. 2.1": "cpc21"})[["hs17_6", "cpc21"]]
codes_cpc21_hs17.hs17_6 = codes_cpc21_hs17.hs17_6.str.replace(".", "")
codes_isic4_cpc21 = pandas.read_csv(
    base_path / "BACI" / "isic4-cpc21.txt", dtype="str"
).rename(columns={"ISIC4code": "isic_rev4_4", "CPC21code": "cpc21"})[
    ["isic_rev4_4", "cpc21"]
]
codes_hs17_isic = codes_cpc21_hs17.merge(codes_isic4_cpc21, on="cpc21", how="left")
codes_hs17_isic["isic_rev4_2"] = codes_hs17_isic.isic_rev4_4.str.slice(stop=2)
codes_hs17_isic["hs17_4"] = codes_hs17_isic.hs17_6.str.slice(stop=4)
codes_hs17_4_isic = (
    codes_hs17_isic.groupby("hs17_4")
    .first()
    .reset_index()
    .drop(columns=["hs17_6", "cpc21"])
)
codes_hs17_isic["hs17_2"] = codes_hs17_isic.hs17_6.str.slice(stop=2)
codes_hs17_2_isic = (
    codes_hs17_isic.groupby("hs17_2")
    .first()
    .reset_index()
    .drop(columns=["hs17_6", "hs17_4", "cpc21", "isic_rev4_4"])
)
codes_hs17_2_isic

In [ ]:
merge_via_cpc_hs4 = df.merge(
    codes_hs17_2_isic, left_on="hs2", right_on="hs17_2", how="left"
)
merge_via_cpc_hs4_success = merge_via_cpc_hs4[
    ~merge_via_cpc_hs4.isic_rev4_2.isna()
].drop(columns=["hs17_2"])
merge_via_cpc_hs4_fail = merge_via_cpc_hs4[merge_via_cpc_hs4.isic_rev4_2.isna()].drop(
    columns=["isic_rev4_2", "hs17_2"]
)

In [ ]:
merged = merge_via_cpc_hs4_success
assert len(merged) == len(df)

In [ ]:
with open(base_path / "BACI" / "ISIC_Rev_4_english_structure.Txt") as fh:
    r = csv.DictReader(fh)
    sector = ""
    letter = ""
    two_digit = ""
    three_digit = ""
    sector = ""
    data = []

    for row in r:
        code = row["Code"]
        desc = row["Description"]
        is_numeric = re.match("^[0-9]+$", code) is not None
        num_digits = len(code)
        if not is_numeric:
            letter = code
            if letter == "A":
                sector = "ag"
            elif letter in ("B", "C", "D", "E", "F"):
                sector = "man"  ## and other industry
            else:
                sector = "serv"
        elif num_digits == 2:
            two_digit = code
        elif num_digits == 3:
            three_digit = code
        else:
            assert num_digits == 4
            data.append(
                {
                    "broad_sector": sector,
                    "isic_rev4_1": letter,
                    "isic_rev4_2": two_digit,
                    "isic_rev4_4": code,
                    "isic_rev4_desc": desc,
                }
            )

isic4_letter = pandas.DataFrame(data)
isic4_letter.to_csv(base_path / "BACI" / "ISIC_Rev_4_english_structure_clean.csv")

In [ ]:
isic4_letter_2 = (
    isic4_letter[["broad_sector", "isic_rev4_1", "isic_rev4_2"]]
    .groupby("isic_rev4_2")
    .first()
)

In [ ]:
merged_with_sector = merged.merge(isic4_letter_2, on="isic_rev4_2", how="left")

In [ ]:
merged_with_sector.columns

In [ ]:
merged_with_sector.to_parquet(
    "../results/input/giri/THA/trade/baci_mode_split_matched_codes.parquet", index=False
)

In [ ]:
grouped = (
    merged_with_sector[
        [
            "iso3_O",
            "iso3_D",
            "broad_sector",
            "v",
            "q",
            "v_air_predict",
            "v_sea_predict",
            "v_land_predict",
            "q_air_predict",
            "q_sea_predict",
            "q_land_predict",
        ]
    ]
    .groupby(["iso3_O", "iso3_D"])
    .sum()
    .reset_index()
)

grouped_sector = (
    merged_with_sector[
        [
            "iso3_O",
            "iso3_D",
            "broad_sector",
            "v",
            "q",
            "v_air_predict",
            "v_sea_predict",
            "v_land_predict",
            "q_air_predict",
            "q_sea_predict",
            "q_land_predict",
        ]
    ]
    .groupby(["iso3_O", "iso3_D", "broad_sector"])
    .sum()
    .reset_index()
)

In [ ]:
tha_trade = grouped.query("iso3_O == 'THA' | iso3_D == 'THA'")
tha_trade_sector = grouped_sector.query("iso3_O == 'THA' | iso3_D == 'THA'")

In [ ]:
tmp = tha_trade_sector.copy()


def get_partner(row):
    if row.iso3_O == "THA":
        return row.iso3_D
    return row.iso3_O


tmp["iso3_partner"] = tmp.apply(get_partner, axis=1)
tmp.groupby(["iso3_partner"]).sum().query("(v > 1e9)")[
    ["v_air_predict", "v_sea_predict", "v_land_predict"]
].plot(kind="bar", title="THA trade mode split (value)", ylabel="Value ('000 USD)")
plt.savefig("../results/input/giri/THA/trade/mode_split_THA_v_partners.png")

In [ ]:
tmp = tha_trade_sector.copy()


def get_partner(row):
    if row.iso3_O == "THA":
        return row.iso3_D
    return row.iso3_O


tmp["iso3_partner"] = tmp.apply(get_partner, axis=1)
tmp.groupby(["iso3_partner"]).sum().query("(q > 1e6)")[
    ["q_air_predict", "q_sea_predict", "q_land_predict"]
].plot(kind="bar", title="THA trade mode split (volume)", ylabel="Volume (metric tons)")
plt.savefig("../results/input/giri/THA/trade/mode_split_THA_q_partners.png")

In [ ]:
for sector in ("ag", "man"):
    tmp = tha_trade_sector.query(
        f"iso3_D != 'THA' & broad_sector == '{sector}' & (v > 1e9)"
    )
    tmp.set_index("iso3_D")[
        ["broad_sector", "v_air_predict", "v_sea_predict", "v_land_predict"]
    ].plot(
        kind="bar",
        title=f"THA '{sector}' sector exports",
        ylabel="Value ('000 USD)",
        xlabel="Destination",
    )

In [ ]:
tmp = tha_trade.query("iso3_O != 'THA' & (v > 1e9)")
tmp.set_index("iso3_O")[["v_air_predict", "v_sea_predict", "v_land_predict"]].plot(
    kind="bar", title="THA imports", ylabel="Value ('000 USD)", xlabel="Source"
)

tmp = tha_trade.query("iso3_D != 'THA' & (v > 1e9)")
tmp.set_index("iso3_D")[["v_air_predict", "v_sea_predict", "v_land_predict"]].plot(
    kind="bar", title="THA exports", ylabel="Value ('000 USD)", xlabel="Destination"
)

In [ ]:
# each way, value
tha_import_split_v = (
    merged_with_sector.query("iso3_D == 'THA'")[
        [
            "broad_sector",
            "v_air_predict",
            "v_sea_predict",
            "v_land_predict",
        ]
    ]
    .groupby(["broad_sector"])
    .sum()
    .reset_index()
)

tha_export_split_v = (
    merged_with_sector.query("iso3_O == 'THA'")[
        [
            "broad_sector",
            "v_air_predict",
            "v_sea_predict",
            "v_land_predict",
        ]
    ]
    .groupby(["broad_sector"])
    .sum()
    .reset_index()
)

# each way, volume
tha_import_split_q = (
    merged_with_sector.query("iso3_D == 'THA'")[
        ["broad_sector", "q_air_predict", "q_sea_predict", "q_land_predict"]
    ]
    .groupby(["broad_sector"])
    .sum()
    .reset_index()
)

tha_export_split_q = (
    merged_with_sector.query("iso3_O == 'THA'")[
        ["broad_sector", "q_air_predict", "q_sea_predict", "q_land_predict"]
    ]
    .groupby(["broad_sector"])
    .sum()
    .reset_index()
)

# either way
tha_trade_split_q = (
    merged_with_sector.query("iso3_O == 'THA' | iso3_D == 'THA'")[
        ["broad_sector", "q_air_predict", "q_sea_predict", "q_land_predict"]
    ]
    .groupby(["broad_sector"])
    .sum()
    .reset_index()
)

tha_trade_split_v = (
    merged_with_sector.query("iso3_O == 'THA' | iso3_D == 'THA'")[
        [
            "broad_sector",
            "v_air_predict",
            "v_sea_predict",
            "v_land_predict",
        ]
    ]
    .groupby(["broad_sector"])
    .sum()
    .reset_index()
)

In [ ]:
tha_import_split_v.set_index("broad_sector").plot(
    kind="bar", title="THA import mode split (value)", ylabel="Value ('000 USD)"
)
tha_export_split_v.set_index("broad_sector").plot(
    kind="bar", title="THA export mode split (value)", ylabel="Value ('000 USD)"
)
tha_import_split_q.set_index("broad_sector").plot(
    kind="bar", title="THA import mode split (volume)", ylabel="Volume (metric tons)"
)
tha_export_split_q.set_index("broad_sector").plot(
    kind="bar", title="THA export mode split (volume)", ylabel="Volume (metric tons)"
)

In [ ]:
tha_trade_split_v.set_index("broad_sector").plot(
    kind="bar", title="THA trade mode split (value)", ylabel="Value ('000 USD)"
)
plt.savefig("../results/input/giri/THA/trade/mode_split_THA_v.png")
tha_trade_split_q.set_index("broad_sector").plot(
    kind="bar", title="THA trade mode split (volume)", ylabel="Volume (metric tons)"
)
plt.savefig("../results/input/giri/THA/trade/mode_split_THA_q.png")

In [ ]:
tha_trade_sector_to_save = tha_trade_sector.rename(
    columns={
        "broad_sector": "sector",
        "iso3_O": "export_country_code",
        "iso3_D": "import_country_code",
        "v": "trade_value_thousandUSD",
        "q": "trade_quantity_tons",
        "v_air_predict": "trade_value_thousandUSD__air",
        "q_air_predict": "trade_quantity_tons__air",
        "v_land_predict": "trade_value_thousandUSD__land",
        "q_land_predict": "trade_quantity_tons__land",
        "v_sea_predict": "trade_value_thousandUSD__sea",
        "q_sea_predict": "trade_quantity_tons__sea",
    }
)

tha_trade_sector_to_save.to_csv(
    "../results/input/giri/THA/trade/baci_sector_trade_THA_from_mode_split.csv",
    index=False,
)
tha_trade_sector_to_save.head()